In [5]:
%pwd #print the current working directory

'c:\\Users\\oanuf\\Google Drive\\Programming\\GitHub\\Alpaca_trading'

In [12]:
import pandas as pd
import datetime as dt

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots


from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
# Importing Alpaca SDK
from Alpaca_config import *

import fmpsdk

import alpaca_trade_api as tradeapi
from alpaca_trade_api.rest import TimeFrame
from alpaca_trade_api.rest import TimeFrameUnit
from pytz import timezone

alpaca = tradeapi.REST(API_KEY_PAPER, API_SECRET_PAPER, API_BASE_URL_PAPER, 'v2')

In [ ]:
hist_index_member=pd.read_excel('D:\\Data\\Other_data\\ETFDB.xlsx',sheet_name='Stock_tickers',skiprows=0,header=1,usecols=['symbol', 'was_us_index_const']) # read hist index components
hist_index_member = hist_index_member[hist_index_member.was_us_index_const=="Yes"].symbol.to_list()
hist_capital = []
for idx, symbol in enumerate(hist_index_member):
    try: # getting hist capitalisation for every symbol in scope
        hist_capital.append(pd.json_normalize(fmpsdk.historical_market_capitalization(apikey=fmp_key, symbol=symbol,limit=3000)))
        print('Done with %s. Still %i to go' % (symbol, len(hist_index_member)-idx-1))
    except:
        pass
all_hist_capital = pd.concat(hist_capital) # putting all in one df
all_hist_capital['date'] = pd.to_datetime(all_hist_capital['date'], format='%Y-%m-%d')
all_hist_capital['date'].min().strftime("%Y-%m-%d") # check the earliest date
all_hist_capital[all_hist_capital.isna().any(axis=1)] # check if there are any N/As
all_hist_capital = all_hist_capital.loc[all_hist_capital['date'] > "2012"] # cut to start from 2012
unique_dates= all_hist_capital['date'].value_counts()
all_hist_capital = all_hist_capital[~all_hist_capital['date'].isin(unique_dates[unique_dates < 5].index)] 
    # there are some outliers 
    # (e.g. available cap for strange tickers on not-working days)
    # => we count which dates appear very few times and delete the rows with these dates
all_hist_capital["rank"] = all_hist_capital.groupby("date")["marketCap"].rank(ascending=False) # rank by cap on every date
all_hist_capital.to_csv('hist_capitalisation_index_constit.csv',index = False)

If the capitalization file was already created, start from reading it:

In [25]:
all_hist_capital = pd.read_csv('D:\\Data\\Other_data\\hist_capitalisation_index_constit.csv',parse_dates=['date'])
all_hist_capital

,symbol,date,marketCap,rank
0,AAPL,2021-12-27,2.958566e+12,1.0
1,AAPL,2021-12-23,2.892120e+12,1.0
2,AAPL,2021-12-22,2.881620e+12,1.0
3,AAPL,2021-12-20,2.784986e+12,1.0
4,AAPL,2021-12-17,2.807791e+12,1.0
...,...,...,...,...
1395984,MBI,2012-01-09,2.356327e+09,471.0
1395985,MBI,2012-01-06,2.389188e+09,470.0
1395986,MBI,2012-01-05,2.362126e+09,469.0
1395987,MBI,2012-01-04,2.342796e+09,469.0


In [26]:

all_hist_capital['date'].min().strftime("%Y-%m-%d") # check the earliest date

'2012-01-03'

In [35]:
fig1 = px.bar(all_hist_capital['symbol'].value_counts(),title="Number of occurence of different tickers from 2012")
fig2 = px.bar(all_hist_capital['date'].value_counts(),title="Number of tickers on every date").update_traces(marker_color='green')
fig1.update_layout(showlegend=False,title_x=0.5)
fig2.update_layout(yaxis_range=[450,600],showlegend=False,title_x=0.5)

In [ ]:
all_hist_capital = all_hist_capital.loc[all_hist_capital['rank'] < 51] # keep only top50 on every date

There are some mistakes in data for capitalization. We need to remove some symbols.

In [ ]:
# delete symbols which appeared by mistake
unique_symb= all_hist_capital['symbol'].value_counts() # counts how many times each symbol appears
check_symb = all_hist_capital[all_hist_capital['symbol'].isin(unique_symb[unique_symb < 10].index)] # select rows, where symbol appears <10 times
check_symb = check_symb[check_symb['rank'] < 48] # if it is on the border than it's ok that it appear too few times
all_hist_capital = all_hist_capital[~all_hist_capital['symbol'].isin(set(check_symb.symbol))] # these symbols we shouldexclude from df as they probably appeared among top50 by mistake

Creating some small dataframe for further experiments. Just limit to top5 by capitalization and daily data from 2019.

In [ ]:
test_capital = all_hist_capital.loc[all_hist_capital['rank'] < 5]
test_capital = test_capital.loc[test_capital['date'] > "2019"]
test_capital['date'].min().strftime("%Y-%m-%d") # check the earliest date
test_capital.to_csv('test_capital.csv')

In [ ]:
minute_frame = 10 # means 1 day is 39 rows => 43k rows for 5 years 
today = test_capital['date'].max().strftime("%Y-%m-%d")
n_days_ago = test_capital['date'].min().strftime("%Y-%m-%d")
historicalData = {}
for symbol in list(set(test_capital.symbol))[0:2]:
    temp = alpaca.get_bars(symbol, TimeFrame(minute_frame, TimeFrameUnit.Minute), n_days_ago, today,adjustment='all').df
    temp.index = temp.index.tz_convert('US/Eastern') # convert to Eastern Time
    temp.index = temp.index.tz_localize(None) # remove +00:00 from datetime
    temp = temp.between_time('09:31', '15:59') # focus on market hours as for now trading on alpaca is restricted to market hours
    temp = temp['close']
    historicalData[symbol]=temp


today = '2021-12-27'
n_days_ago = '2021-12-17'
temp = alpaca.get_bars('AMZN', TimeFrame(minute_frame, TimeFrameUnit.Minute), n_days_ago, today,adjustment='all').df
temp.index = temp.index.tz_convert('US/Eastern') # convert to Eastern Time
temp.index = temp.index.tz_localize(None) # remove +00:00 from datetime
temp['chg'] = round(((temp.close - temp.open)/temp.open) * 100,2)
temp.nlargest(20,'chg')
temp["time"] = temp.index.time
temp["date"] = temp.index.date
temp = temp.between_time('09:31', '15:59') # focus on market hours as for now trading on alpaca is restricted to market hours
temp['time'] = temp['time'].dt.floor('Min')

newf = temp[['close','time','date']]
newf = newf.pivot(index='date', columns='time')
temp.columns


all_hist_capital.resample('W', on='date').mean()
all_hist_capital['symbol'].value_counts()
all_hist_capital['date'].value_counts()


fig = px.bar(all_hist_capital['date'].value_counts())
fig = px.bar(temp, y="trade_count")
fig.show()


